In [ ]:
import os
import train_model
from aws_helpers import create_repository, setup_elb, create_ecs_service, deregister_old_taskdefinitions
from keras_to_tfserving import convert_keras_to_tf_model

## Requirements
This code requires a couple of things:
- Installed requirements
- Installed Docker (https://docs.docker.com/install/linux/docker-ce/ubuntu/)
- Installed & configured AWS CLI (https://aws.amazon.com/cli/)
- AWS Elastic Load Balancer setup as an Application Load Balancer with a listener on 443 (and an SSL certificate)
- AWS Elastic Container Service cluster setup and configured

## Setting up a new Service

In [ ]:
version = 1
service_path = 'services.dev.sellpy.net'
service_name = "stockholm-ai"
cluster = "microservices"
env = "dev"
load_balancer_name = "sellpy-services"

### Train model & Convert model

In [ ]:
model_name = "stockholm_ai_mnist.m"  

if model_name not in os.listdir("."):
    train_model.output_model(model_name, epochs=1)

!python3 keras_to_tfserving.py stockholm_ai_mnist.m models/stockholm-ai

# TODO: Make this repeatable in notebook
#convert_keras_to_tf_model(model_name=model_name,
#                          model_path="models/stockholm-ai", # Note that the model path is referred to in server.conf
#                          sequential=True)

### Create a repository on ECR, build & push the image

In [ ]:
# Create an image repository for storing docker files.
create_repository(service_name)

# Shellscript.
# could write it like: subprocess.call("<command>", shell=True)
!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev


### Register a target & listener in the load balancer

In [ ]:
target_group_arn = setup_elb(
    service=service_name,
    version=version,
    load_balancer_name=load_balancer_name,
    service_path=service_path
)

### Register a task definition and create an ECS service

In [ ]:
# Create / Update a task definition.
!aws ecs register-task-definition --cli-input-json file://task_definition.json --region eu-west-1

create_ecs_service(
    cluster=cluster,
    service=service_name,
    task_definition_family=service_name,
    target_group_arn=target_group_arn,
    service_count=1
)

deregister_old_taskdefinitions(service_name)

## Update existing services

In [ ]:
#rebuild image
!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev

# Register a new Task def 
!aws ecs register-task-definition --cli-input-json file://task_definition.json --region eu-west-1
# Update the service with a new task def
!aws ecs update-service --service stockholm-ai --task-definition stockholm-ai --region eu-west-1  --cluster microservices
# Deregister tasks
deregister_old_taskdefinitions(service_name)